In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from shapely.geometry import Point
import geopandas as gpd
import copy
import networkx as nx

### Dataset for prediction
Each observation/row is a connection (edge in the network). 
We want the following **route-specific** information: 
* Number of flights ("count") **done**
* Distance (average distance?) **done**
* Tid (average time?)  **done** 


We want the following **airport-specific** information: 
* Degree **done** 
* Clustering coefficient **done**
* Betweenness centrality **done** 
* State? (administrative border?) **excluded**


## Load data 
Data comes from http://stat-computing.org/dataexpo/2009/. 
Flight data relates to the year 2007 (see file StatComp_MER) 

In [2]:
# Load data
Flights = pd.read_pickle("Flights.pkl")
#Airports = pd.read_pickle("airports.pkl")

# Select variables from which to create network. Must contain two columns of nodes, since 
# nx.from_pandas_edgelist() views each row as an edge. Can also contains attributes, e.g. Distance. 
FlightsNx = Flights[["Origin","Dest","Distance"]]

# Take counts (i.e. how many origin to destination flights in the dataset)
counts = pd.DataFrame(Flights.groupby(["Origin","Dest"])["Year"].count())
counts.columns = ["count"]
FlightsNx = FlightsNx.merge(counts, how = 'left', on = ["Origin","Dest"])

# Drop duplicates 
FlightsNx = FlightsNx.drop_duplicates()

In [3]:
# Create average time for each route
TimeMeans = pd.DataFrame(Flights.groupby(["Origin","Dest"])["ActualElapsedTime"].mean())
# Merge to FlightsNx
FlightsNx = FlightsNx.merge(TimeMeans, how = "left", on = ["Origin", "Dest"])

In [4]:
# Create number of distinct companies on each route 
companies = pd.DataFrame(Flights.groupby(["Origin","Dest"])["UniqueCarrier"].nunique())
companies.columns = ["companies"]
FlightsNx = FlightsNx.merge(companies, how = "left", on = ["Origin", "Dest"])

In [5]:
# Count number of take-offs from origin: 
orig_takeoff = pd.DataFrame(FlightsNx.groupby(["Origin"])["count"].sum())
orig_takeoff.columns = ["origin_takeoff"]
# Merge to network dataset: 
FlightsNx = FlightsNx.merge(orig_takeoff, how = "left", on = "Origin")

# Count number of take-offs from dest
dest_takeoff = pd.DataFrame(FlightsNx.groupby(["Origin"])["count"].sum())
dest_takeoff.columns = ["dest_takeoff"]
FlightsNx = FlightsNx.merge(dest_takeoff, how = "left", left_on = "Dest", right_on = "Origin")

# Count number of landings at destination
dest_lands = pd.DataFrame(FlightsNx.groupby(["Dest"])["count"].sum())
dest_lands.columns = ["dest_landing"]
FlightsNx = FlightsNx.merge(dest_lands, how = "left", on = "Dest")

orig_lands = pd.DataFrame(FlightsNx.groupby(["Dest"])["count"].sum())
orig_lands.columns = ["orig_landing"]
FlightsNx = FlightsNx.merge(orig_lands, how = "left", left_on = "Origin", right_on = "Dest")

# Construct totals (flights)
FlightsNx["Origin_flights"] = FlightsNx["origin_takeoff"]+FlightsNx["orig_landing"]
FlightsNx["Destination_flights"] = FlightsNx["dest_takeoff"]+FlightsNx["dest_landing"]

# Drop auxiliary vars
FlightsNx.drop(["origin_takeoff","dest_takeoff","dest_landing","orig_landing"], axis = 1, inplace = True)

### Degree distribution

In [6]:
Network = nx.from_pandas_edgelist(FlightsNx, source = "Origin", target = "Dest", edge_attr = True)
# docs: https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.centrality.degree_centrality.html#networkx.algorithms.centrality.degree_centrality 
# Find degree: 
degree_dict = nx.algorithms.centrality.degree_centrality(Network) 
degree_values = list(degree_dict.values())
# Create dataframe
degrees = pd.DataFrame(degree_values)
degrees.columns = ["degree"]
degrees["degree_unnormalized"] = degrees["degree"]*(len(Network.nodes)-1)
degrees["airport"] = degree_dict.keys()

In [7]:
# Add degree to FlightsNx data
FlightsNx = FlightsNx.merge(degrees, how = "left", left_on = "Origin", right_on = "airport")
FlightsNx = FlightsNx.drop("airport", axis = 1)
FlightsNx = FlightsNx.rename(columns = {"degree": "origin_degree", "degree_unnormalized": "origin_degree_unn"})

FlightsNx = FlightsNx.merge(degrees, how = "left", left_on = "Dest", right_on = "airport")
FlightsNx = FlightsNx.drop("airport", axis = 1)
FlightsNx = FlightsNx.rename(columns = {"degree": "dest_degree", "degree_unnormalized": "dest_degree_unn"})


#### Betweenness centrality

In [8]:
btwn_dict = nx.betweenness_centrality(Network)
btwns = pd.DataFrame(list(btwn_dict.values()))
btwns.columns = ["betweenness"]
btwns["airport"] = btwn_dict.keys()

FlightsNx = FlightsNx.merge(btwns, how = "left", left_on = "Origin", right_on = "airport")
FlightsNx = FlightsNx.drop("airport", axis = 1)
FlightsNx = FlightsNx.rename(columns = ({"betweenness":"origin_btwns"}))

FlightsNx = FlightsNx.merge(btwns, how = "left", left_on = "Dest", right_on = "airport")
FlightsNx = FlightsNx.drop("airport", axis = 1)
FlightsNx = FlightsNx.rename(columns = ({"betweenness":"dest_btwns"}))

In [9]:
FlightsNx.drop(["origin_degree","dest_degree"], axis = 1, inplace = True)

In [10]:
FlightsNx = FlightsNx.rename(columns = ({"ActualElapsedTime":"avg_time_mins",
                             "origin_degree_unn":"origin_degree",
                             "dest_degree_unn":"dest_degree"}))

#### Clustering Coefficients 

In [11]:
clusters = nx.algorithms.cluster.clustering(Network)
clusters_df = pd.DataFrame(list(clusters.values()))
clusters_df.columns = ["clustering"]
clusters_df["airport"] = clusters.keys()

FlightsNx = FlightsNx.merge(clusters_df, how = "left", left_on = "Origin", right_on = "airport")
FlightsNx = FlightsNx.drop("airport", axis = 1)
FlightsNx = FlightsNx.rename(columns = ({"clustering":"origin_clustcoef"}))

FlightsNx = FlightsNx.merge(clusters_df, how = "left", left_on = "Dest", right_on = "airport")
FlightsNx = FlightsNx.drop("airport", axis = 1)
FlightsNx = FlightsNx.rename(columns = ({"clustering":"dest_clustcoef"}))


In [12]:
FlightsNx.to_pickle("FlightsML.pkl")
FlightsNx.head()

,Origin,Dest,Distance,count,avg_time_mins,companies,Origin_flights,Destination_flights,origin_degree,dest_degree,origin_btwns,dest_btwns,origin_clustcoef,dest_clustcoef
0,SMF,ONT,389,4093,71.234185,1,116158,83287.0,44.0,40.0,0.011491,0.003241,0.486258,0.597436
1,SMF,PDX,479,2721,83.317127,2,116158,117237.0,44.0,43.0,0.011491,0.003647,0.486258,0.591362
2,SMF,PHX,647,4184,106.243087,3,116158,422170.0,44.0,88.0,0.011491,0.028447,0.486258,0.375392
3,SMF,SAN,480,4895,81.599257,2,116158,194618.0,44.0,53.0,0.011491,0.002061,0.486258,0.593614
4,SMF,SEA,605,3519,106.077896,2,116158,218274.0,44.0,61.0,0.011491,0.044200,0.486258,0.475410


In [13]:
## Homophily? 
## " Does these features contribute? i.e. raise predictive performance?"
## Number of different companies?